# Municipal Crash Data Aggregation 
## The function of this Jupyter notebook is to collect, query and merge Municipal crash, ridership, government, demographics, and other important data into one data frame to compare and analyze in another notebook. 


### Sources 

#### DATA In Use
Penn DOT Crash Data https://pennshare.maps.arcgis.com/apps/webappviewer/index.html?id=8fdbf046e36e41649bbfd9d7dd7c7e7e 
click on here hyperlink to obtain dictionary



Pennsylvania Government Data (Land Area, Government Form, Municipal Name, Number of Employees)https://munstats.pa.gov/Reports/ReportInformation2.aspx?report=CountyMuniDemo_Excel


Census API discovery tool https://www.census.gov/data/developers/updates/new-discovery-tool.html

#### Out of Use
ACS Municipal data https://data.pa.gov/w/sw4q-ahf4/33ch-zxdi?cur=hzGbeDpR0U7&from=FN6e8L6mmBt



### Tutorials Used

In [1]:
#import necessary libraries 
#

import requests
import json, csv
import timeit
from pathlib import Path  

# import data handling libraries 
import numpy as np
import pandas as pd
import requests
import json
#from dateutil.rrule import rrule, DAILY, MONTHLY
#from datetime import  timedelta


#import graphing libraries 
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors




from pathlib import Path  



pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:,.6f}'.format

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Set Global Variables

In [2]:
#Set Global Variables

# set the time period in which to look at 

START_YEAR=2017
END_YEAR=2021




# Querry American Community Survey(ACS) 

https://api.census.gov/data/acs/acs5/get='NAME','B01001_001E','B08006_014E','B08006_014M','B08006_015E','B08006_015M','B08006_003E','B08006_003M','B08006_004E','B08006_004M','B08006_008E','B08006_008M'&for=county subdivision:*&in=state:42

In [3]:
# Query ACS data for Municipalities 

#Census API
HOST = 'https://api.census.gov/data'

#Year to get data for 
YEAR = '{}'.format(END_YEAR)#,2005,2010,2015,2020'  

# Survey to Access data from (ACS 5 year estimates)
DATA_SET = 'acs/acs5'
BASE_URL = '/'.join([HOST, YEAR, DATA_SET])

#create an empty dictionary for predicates 
predicates = {}

# VARIABLES
# Population Estimate:  B01003_001E
# Population Margin of error: B01003_001M
# Bike to work Estimate: B08006_014E
# Bike to work Margin of error: B08006_014M
# Walk to work Estimate:B08006_015E
# Walk to work Margin of error:B08006_015M
# Drive to work alone Estimate: B08006_003E
# Drive to work alone Margin of error: B08006_003M
# Carpool to work Estimate:B08006_004E
# Carpool to work Margin: 
# Public transit to work Estimate: B08006_008E
# Public transit to work Margine of Error: B08006_008E
# Other to Work 
# Poverty Estimate:
# Poverty Margin of error:

get_vars = ['NAME',
            'B01001_001E',
            'B08006_014E',
            'B08006_014M',
            'B08006_015E',
            'B08006_015M',
            'B08006_003E',
            'B08006_003M',
            'B08006_004E',
            'B08006_004M',
            'B08006_008E',
            'B08006_008M']

predicates['get']=','.join(get_vars)

#Set sub geographies to get data for ('county','Place','county subdivision) * means get all 
predicates['for']='county subdivision:*'

# Set geography that contains sub geographies  (42=Pennslyvania )
predicates['in']='state:42'

#assemble the API query 
r = requests.get(BASE_URL, params = predicates)

#print the query URL 
print(BASE_URL,predicates)


https://api.census.gov/data/2021/acs/acs5 {'get': 'NAME,B01001_001E,B08006_014E,B08006_014M,B08006_015E,B08006_015M,B08006_003E,B08006_003M,B08006_004E,B08006_004M,B08006_008E,B08006_008M', 'for': 'county subdivision:*', 'in': 'state:42'}


In [4]:
#print the length and first thousand characters to see what you got 
print(len(r.text))
print(r.text[0:1000])


334430
[["NAME","B01001_001E","B08006_014E","B08006_014M","B08006_015E","B08006_015M","B08006_003E","B08006_003M","B08006_004E","B08006_004M","B08006_008E","B08006_008M","state","county","county subdivision"],
["Abbottstown borough, Adams County, Pennsylvania","857","7","11","6","8","355","60","43","22","0","11","42","001","00116"],
["Arendtsville borough, Adams County, Pennsylvania","720","0","11","4","9","301","92","41","33","0","11","42","001","02928"],
["Bendersville borough, Adams County, Pennsylvania","672","0","11","4","5","305","97","6","9","0","11","42","001","05536"],
["Berwick township, Adams County, Pennsylvania","2495","0","11","5","6","961","146","129","64","0","11","42","001","05880"],
["Biglerville borough, Adams County, Pennsylvania","1209","0","11","20","24","471","105","86","96","0","11","42","001","06296"],
["Bonneauville borough, Adams County, Pennsylvania","1798","0","11","5","6","867","163","72","57","0","11","42","001","07560"],
["Butler township, Adams County, 

In [5]:
#Place the Queried ACS data into a data frame 

# set the column names to the first row of data from the query 
column_names=r.json()[0:1][0]

#Set the data to everything after the first row and convert to an array to flatten it
ACS_DATA= r.json()[1:]
ACS_data = np.array(ACS_DATA)

#create the pandas data frame 
ACS_MUNI_DF = pd.DataFrame(columns=column_names , data = ACS_data)

# reset the index of the data frame
ACS_MUNI_DF.reset_index()

# rename the columns 
ACS_MUNI_DF.rename(columns ={"B01001_001E":"POPULATION",
                    "B08006_014E":'BIKE_TO_WORK_EST',
                    "B08006_014M":"BIKE_TO_WORK_MARG",
                    "B08006_015E":"WALK_TO_WORK_EST",
                    "B08006_015M":"WALK_TO_WORK_MARG",
                    'B08006_003E':"DRIVE_SOLO_TO_WORK_EST",
                    'B08006_003M':"DRIVE_SOLO_TO_WORK_MARG",
                    'B08006_004E':"CARPOOL_TO_WORK_EST",
                    'B08006_004M':"CARPOOL_TO_WORK_MARG",
                    'B08006_008E':"PUBTRANS_TO_WORK_EST",
                    'B08006_008M':"PUBTRANS_TO_WORK_MARG",
                    "county subdivision":"county_subdivision"}, inplace=True)

#convert the NAME column to strings 
ACS_MUNI_DF['NAME'] = ACS_MUNI_DF["NAME"].astype(str)

# remove Pennsylvania from NAME This is over redundant since all data will be from Pennsylvania 
ACS_MUNI_DF['NAME'] = ACS_MUNI_DF.NAME.replace({', Pennsylvania':''}, regex=True)

#create separate Name variables for county and municipality 
ACS_MUNI_DF[['MUNI_NAME','COUNTY_NAME']]=ACS_MUNI_DF['NAME'].str.split(',',expand=True)

# convert the variables to integers
ACS_MUNI_DF[["POPULATION",
                "BIKE_TO_WORK_EST",
                "BIKE_TO_WORK_MARG",
                "WALK_TO_WORK_EST",
                "WALK_TO_WORK_MARG",
                "DRIVE_SOLO_TO_WORK_EST",
                "DRIVE_SOLO_TO_WORK_MARG",
                "CARPOOL_TO_WORK_EST",
                "CARPOOL_TO_WORK_MARG",
                "PUBTRANS_TO_WORK_EST",
                "PUBTRANS_TO_WORK_MARG",
                "state",                 
                "county",                
                "county_subdivision"]] = ACS_MUNI_DF[["POPULATION",
                                            "BIKE_TO_WORK_EST",
                                            "BIKE_TO_WORK_MARG",
                                            "WALK_TO_WORK_EST",
                                            "WALK_TO_WORK_MARG",
                                            "DRIVE_SOLO_TO_WORK_EST",
                                            "DRIVE_SOLO_TO_WORK_MARG",
                                            "CARPOOL_TO_WORK_EST",
                                            "CARPOOL_TO_WORK_MARG",
                                            "PUBTRANS_TO_WORK_EST",
                                            "PUBTRANS_TO_WORK_MARG",
                                            "state",                 
                                            "county",                
                                            "county_subdivision"]].astype(int)
print(len(ACS_MUNI_DF))
ACS_MUNI_DF.head(3)


2573


,NAME,POPULATION,BIKE_TO_WORK_EST,BIKE_TO_WORK_MARG,WALK_TO_WORK_EST,WALK_TO_WORK_MARG,DRIVE_SOLO_TO_WORK_EST,DRIVE_SOLO_TO_WORK_MARG,CARPOOL_TO_WORK_EST,CARPOOL_TO_WORK_MARG,PUBTRANS_TO_WORK_EST,PUBTRANS_TO_WORK_MARG,state,county,county_subdivision,MUNI_NAME,COUNTY_NAME
0,"Abbottstown borough, Adams County",857,7,11,6,8,355,60,43,22,0,11,42,1,116,Abbottstown borough,Adams County
1,"Arendtsville borough, Adams County",720,0,11,4,9,301,92,41,33,0,11,42,1,2928,Arendtsville borough,Adams County
2,"Bendersville borough, Adams County",672,0,11,4,5,305,97,6,9,0,11,42,1,5536,Bendersville borough,Adams County


In [6]:
len(ACS_MUNI_DF)


2573

In [7]:
ACS_MUNI_DF[ACS_MUNI_DF["MUNI_NAME"]=="Carlisle borough"].head()

,NAME,POPULATION,BIKE_TO_WORK_EST,BIKE_TO_WORK_MARG,WALK_TO_WORK_EST,WALK_TO_WORK_MARG,DRIVE_SOLO_TO_WORK_EST,DRIVE_SOLO_TO_WORK_MARG,CARPOOL_TO_WORK_EST,CARPOOL_TO_WORK_MARG,PUBTRANS_TO_WORK_EST,PUBTRANS_TO_WORK_MARG,state,county,county_subdivision,MUNI_NAME,COUNTY_NAME
962,"Carlisle borough, Cumberland County",19869,235,104,1052,222,6598,481,547,183,42,37,42,41,11272,Carlisle borough,Cumberland County


In [8]:
ACS_MUNI_DF["NAME"][ACS_MUNI_DF["POPULATION"].isna()==True].unique()

array([], dtype=object)

# Import and Clean Penn DoT's County and Municipal Codes 
##### Dictionary Source: 
the identifying numbers were extracted from dictionary for
Penn DOT's Crash Data https://pennshare.maps.arcgis.com/apps/webappviewer/index.html?id=8fdbf046e36e41649bbfd9d7dd7c7e7e click on the dictionary hyperlink once you open the page


In [9]:
#import Penn DoT's County code CSV

#Location:
#/home/mitch/Data_Work/data_folder/Municipal_data/Penndot_County_Name&Number.csv

#import data
PENNDOT_COUNTY_DF = pd.read_csv('../../../Data_Work/data_folder/Municipal_data/Penndot_County_Name&Number.csv')

#delete the first two rows since they are empty
PENNDOT_COUNTY_DF=PENNDOT_COUNTY_DF[2:]

#rename the Columns 
PENNDOT_COUNTY_DF.rename(columns={'Unnamed: 0':'PENN_DOT_COUNTY_NUM', 'Unnamed: 1':'PENN_DOT_COUNTY_NAME'}, inplace=True)

# add county string onto the county name use an if function to prevent duplicate string additions
COUNTY_LIST=[]
COUNTY_LIST = PENNDOT_COUNTY_DF['PENN_DOT_COUNTY_NAME'].tolist()
New_County_List=[]
for county in COUNTY_LIST:
    if county[-8:]!=' COUNTY':
         county = county+' COUNTY'
    else:None
    New_County_List.append(county)

PENNDOT_COUNTY_DF['PENN_DOT_COUNTY_NAME']=New_County_List

#fix teh index
PENNDOT_COUNTY_DF.reset_index(inplace=True)
PENNDOT_COUNTY_DF.drop(columns=('index'), inplace=True)

#print the length
print(len(PENNDOT_COUNTY_DF))
PENNDOT_COUNTY_DF.head()

67


,PENN_DOT_COUNTY_NUM,PENN_DOT_COUNTY_NAME
0,1,ADAMS COUNTY
1,2,ALLEGHENY COUNTY
2,3,ARMSTRONG COUNTY
3,4,BEAVER COUNTY
4,5,BEDFORD COUNTY


In [10]:
#Import Penn DoT's Municipal Codes

#Location:
#/home/mitch/Data_Work/data_folder/Municipal_data/PennDOT_MUNI_CODE2.csv


#municipalities that may be problematic :
#40304 - Wilkes-Barre City
#40104 - Wilkes-Barre Twp

#import PENN Dot Municipal IDs originally taken from data dictionary 
PENNDOT_MUNI_DF = pd.read_csv('../../../Data_Work/data_folder/Municipal_data/PennDOT_MUNI_CODE2.csv')

# Strip white space, Capitalize and set to string  the Municipal NAME for merging 
PENNDOT_MUNI_DF['MUNICIPALITY_NAME'] = PENNDOT_MUNI_DF['MUNICIPALITY_NAME'].str.strip()
PENNDOT_MUNI_DF['MUNICIPALITY_NAME'] = PENNDOT_MUNI_DF['MUNICIPALITY_NAME'].str.upper()
PENNDOT_MUNI_DF['MUNICIPALITY_NAME'] = PENNDOT_MUNI_DF['MUNICIPALITY_NAME'].astype(str)

#replace abriviated boro and township with full spelling  
PENNDOT_MUNI_DF['MUNICIPALITY_NAME'] = PENNDOT_MUNI_DF.MUNICIPALITY_NAME.replace({' BORO':' BOROUGH'}, regex=True)
PENNDOT_MUNI_DF['MUNICIPALITY_NAME'] = PENNDOT_MUNI_DF.MUNICIPALITY_NAME.replace({' TWP':' TOWNSHIP'}, regex=True)

PENNDOT_MUNI_DF.rename(columns={'MUNICIPALITY_NAME':'PENN_DOT_MUNI_NAME','MUNI_ID':'PENN_DOT_MUNI_ID'}, inplace=True)


#create a column for county IDs from the first two numbers in the 5 digit ID 
PENNDOT_MUNI_DF['PENN_DOT_COUNTY_NUM'] = (PENNDOT_MUNI_DF['PENN_DOT_MUNI_ID']-(PENNDOT_MUNI_DF['PENN_DOT_MUNI_ID']%1000))/1000
PENNDOT_MUNI_DF['PENN_DOT_COUNTY_NUM'] = PENNDOT_MUNI_DF['PENN_DOT_COUNTY_NUM'].astype(int)

print(len(PENNDOT_MUNI_DF))
PENNDOT_MUNI_DF.head()

2584


,PENN_DOT_MUNI_ID,PENN_DOT_MUNI_NAME,PENN_DOT_COUNTY_NUM
0,1201,BERWICK TOWNSHIP,1
1,1202,BUTLER TOWNSHIP,1
2,1203,CONEWAGO TOWNSHIP,1
3,1204,CUMBERLAND TOWNSHIP,1
4,1205,FRANKLIN TOWNSHIP,1


# Import Municipal Government Operations Data  



In [11]:
# import CSV to data frame 

# open the CSV containing Pennsylvania Municipal data (Area and populations and Muni government opperations data)
# https://munstats.pa.gov/Reports/ReportInformation2.aspx?report=CountyMuniDemo_Excel
Muni_GOV_STATS = pd.read_csv('../../../Data_Work/data_folder/Municipal_data/CountyMuniDemo_Excel.csv')

Muni_GOV_STATS['MUNICIPALITY_NAME'] = Muni_GOV_STATS['MUNICIPALITY_NAME'].replace({" BORO":" BOROUGH",
                                                                                   " TWP":" TOWNSHIP"},regex=True)

#Muni_GOV_STATS['MUNICIPALITY_NAME'].drop()

Muni_GOV_STATS['COUNTY_NAME'] = Muni_GOV_STATS['COUNTY_NAME'] +" COUNTY"

print(len(Muni_GOV_STATS))
Muni_GOV_STATS.head()



2635


,COUNTY_NAME,textbox5,MUNICIPALITY_NAME,CENSUS_POPULATION,LAND_AREA,FEDERAL_EIN_CODE,HOME_RULE_CODE,HOME_RULE_NAME,HOME_RULE_YEAR,GOVERNMENTAL_FORM,EMPLOYEES_FULL_TIME,EMPLOYEES_PART_TIME,INCORPORATION_YEAR
0,ADAMS COUNTY,10033,ABBOTTSTOWN BOROUGH,1022,0.600000,236050408,NaN,NaN,NaN,NaN,1.000000,10.000000,"1,835.000000"
1,ADAMS COUNTY,10001,ADAMS COUNTY,103852,522.200000,236003030,NaN,NaN,NaN,NaN,448.000000,62.000000,NaN
2,ADAMS COUNTY,10063,ARENDTSVILLE BOROUGH,867,0.800000,231603459,NaN,NaN,NaN,NaN,3.000000,2.000000,"1,896.000000"
3,ADAMS COUNTY,10093,BENDERSVILLE BOROUGH,736,0.500000,237447802,NaN,NaN,NaN,NaN,0.000000,19.000000,"1,866.000000"
4,ADAMS COUNTY,10125,BERWICK TOWNSHIP,2403,7.700000,232030284,NaN,NaN,NaN,NaN,4.000000,0.000000,"1,800.000000"


# Merge County and Municipal 

In [12]:
# Merge county and Municipal Labels


#create a new data frame for merging county municipal labels so that columns arn't duplicated every time this code is run 
#NEW_PENNDOT_MUNI_DF = pd.merge(PENNDOT_MUNI_DF, PENNDOT_COUNTY_DF, how= 'inner', on='PENN_DOT_COUNTY_NUM')


# Merge Penn DoT Labels and Municipal Government stats with ACS data 

In [13]:
# Merge PennDoT County Labels with ACS data 
# Make the county names Uppercase for the ACS data so that they can be merged with the Penn DoT IDs
ACS_MUNI_DF['COUNTY_NAME'] = ACS_MUNI_DF['COUNTY_NAME'].str.upper()

# Remove white space at beginning and end of ACS county lables 
ACS_MUNI_DF['COUNTY_NAME'] = ACS_MUNI_DF['COUNTY_NAME'].str.strip()

# create new data frame to hold values so that 
COUNTY_ACS_MUNI_DF = ACS_MUNI_DF.merge(PENNDOT_COUNTY_DF, how='left', left_on='COUNTY_NAME', right_on='PENN_DOT_COUNTY_NAME')
print(len(COUNTY_ACS_MUNI_DF))
COUNTY_ACS_MUNI_DF.head(2)


2573


,NAME,POPULATION,BIKE_TO_WORK_EST,BIKE_TO_WORK_MARG,WALK_TO_WORK_EST,WALK_TO_WORK_MARG,DRIVE_SOLO_TO_WORK_EST,DRIVE_SOLO_TO_WORK_MARG,CARPOOL_TO_WORK_EST,CARPOOL_TO_WORK_MARG,PUBTRANS_TO_WORK_EST,PUBTRANS_TO_WORK_MARG,state,county,county_subdivision,MUNI_NAME,COUNTY_NAME,PENN_DOT_COUNTY_NUM,PENN_DOT_COUNTY_NAME
0,"Abbottstown borough, Adams County",857,7,11,6,8,355,60,43,22,0,11,42,1,116,Abbottstown borough,ADAMS COUNTY,1,ADAMS COUNTY
1,"Arendtsville borough, Adams County",720,0,11,4,9,301,92,41,33,0,11,42,1,2928,Arendtsville borough,ADAMS COUNTY,1,ADAMS COUNTY


In [14]:
#create new datafram to merge ACS and Penn DoT labels into



# clean and prepare the ACS columns that will be used to merge the Penn DoT labels and ACS data
COUNTY_ACS_MUNI_DF['PENN_DOT_COUNTY_NUM'] = COUNTY_ACS_MUNI_DF['PENN_DOT_COUNTY_NUM'].astype(int)
COUNTY_ACS_MUNI_DF['MUNI_NAME'] = COUNTY_ACS_MUNI_DF['MUNI_NAME'].str.strip()
COUNTY_ACS_MUNI_DF['MUNI_NAME'] = COUNTY_ACS_MUNI_DF['MUNI_NAME'].str.upper()
COUNTY_ACS_MUNI_DF['MUNI_NAME'] = COUNTY_ACS_MUNI_DF['MUNI_NAME'].replace({"ST. ":"SAINT "},regex=True)

TYPO_TOWNSHIP_NAMES ={"MONROEVILLE MUNICIPALITY":"MONROEVILLE BOROUGH",
            "O'HARA TOWNSHIP" :"OHARA TOWNSHIP",
            "COUNTY SUBDIVISIONS NOT DEFINED":"",
            "SUGARCREEK TOWNSHIP":"SUGAR CREEK TOWNSHIP",
            "TIONESAINT TOWNSHIP":"TIONESTA TOWNSHIP",
            "TIONESAINT BOROUGH":"TIONESTA BOROUGH",
            "VALLEY-HI BOROUGH":"VALLEY HI BOROUGH",
            #"STONYCREEK TOWNSHIP":"STONYCREEK TOWNSHIP",
            "BLACK LICK TOWNSHIP":"BLACK TOWNSHIP",
            "CHERRYTREE TOWNSHIP":"CHERRY TREE TOWNSHIP",
            "FRENCHCREEK TOWNSHIP" :"FRENCH CREEK TOWNSHIP",
            "OIL CITY CITY":"OIL CITY",
            "PINEGROVE TOWNSHIP" :"PINE GROVE TOWNSHIP",
            "SANDYCREEK TOWNSHIP":"SANDY CREEK TOWNSHIP",
             "NANTY-GLO BOROUGH":"NANTY GLO BOROUGH",
             "LATROBE BOROUGH":"LATROBE CITY",
             "MURRYSVILLE MUNICIPALITY":"MUNICIPALITY OF MURRYSVILLE",
             "BLOOMSBURG TOWN":"TOWN OF BLOOMSBURG",
             "BETHEL PARK MUNICIPALITY":"BETHEL PARK BOROUGH",
             "LOWER AUGUSAINT TOWNSHIP":"LOWER AUGUSTA TOWNSHIP",
             "UPPER AUGUSAINT TOWNSHIP":"UPPER AUGUSTA TOWNSHIP"}


for key in TYPO_TOWNSHIP_NAMES:
    COUNTY_ACS_MUNI_DF['MUNI_NAME'] = COUNTY_ACS_MUNI_DF['MUNI_NAME'].str.replace(key,TYPO_TOWNSHIP_NAMES[key])

# Merge the Penn DoT Labels with the the ACS data
ACS_PENNDOT_MUNI_DF =  COUNTY_ACS_MUNI_DF.merge(PENNDOT_MUNI_DF,
                                how='left',
                                left_on=['MUNI_NAME','PENN_DOT_COUNTY_NUM'],
                                right_on=['PENN_DOT_MUNI_NAME','PENN_DOT_COUNTY_NUM'])

print(len(ACS_PENNDOT_MUNI_DF))
ACS_PENNDOT_MUNI_DF.head(2)


2574


,NAME,POPULATION,BIKE_TO_WORK_EST,BIKE_TO_WORK_MARG,WALK_TO_WORK_EST,WALK_TO_WORK_MARG,DRIVE_SOLO_TO_WORK_EST,DRIVE_SOLO_TO_WORK_MARG,CARPOOL_TO_WORK_EST,CARPOOL_TO_WORK_MARG,PUBTRANS_TO_WORK_EST,PUBTRANS_TO_WORK_MARG,state,county,county_subdivision,MUNI_NAME,COUNTY_NAME,PENN_DOT_COUNTY_NUM,PENN_DOT_COUNTY_NAME,PENN_DOT_MUNI_ID,PENN_DOT_MUNI_NAME
0,"Abbottstown borough, Adams County",857,7,11,6,8,355,60,43,22,0,11,42,1,116,ABBOTTSTOWN BOROUGH,ADAMS COUNTY,1,ADAMS COUNTY,"1,401.000000",ABBOTTSTOWN BOROUGH
1,"Arendtsville borough, Adams County",720,0,11,4,9,301,92,41,33,0,11,42,1,2928,ARENDTSVILLE BOROUGH,ADAMS COUNTY,1,ADAMS COUNTY,"1,402.000000",ARENDTSVILLE BOROUGH


In [15]:
#print Municipalities with errors
ACS_PENNDOT_MUNI_DF['MUNI_NAME'][ACS_PENNDOT_MUNI_DF['PENN_DOT_MUNI_ID'].isna()==True].unique()

#two stony creek townships in PA

array(['LERAYSVILLE BOROUGH', '', 'BLACK TOWNSHIP', 'S.N.P.J. BOROUGH',
       'STONYCREEK TOWNSHIP'], dtype=object)

In [16]:
MUNI_DF = ACS_PENNDOT_MUNI_DF.merge(Muni_GOV_STATS, how="left", left_on=["MUNI_NAME","PENN_DOT_COUNTY_NAME"], right_on=["MUNICIPALITY_NAME","COUNTY_NAME"])

print("length of ACS_PENNDOT_MUNI_DF: {}\n".format(len(ACS_PENNDOT_MUNI_DF)))
print("length of Muni_GOV_STATS: {}\n".format(len(Muni_GOV_STATS)))
print("length of MUNI_DF: {}\n".format(len(MUNI_DF)))

MUNI_DF.head(3)


length of ACS_PENNDOT_MUNI_DF: 2574

length of Muni_GOV_STATS: 2635

length of MUNI_DF: 2574



,NAME,POPULATION,BIKE_TO_WORK_EST,BIKE_TO_WORK_MARG,WALK_TO_WORK_EST,WALK_TO_WORK_MARG,DRIVE_SOLO_TO_WORK_EST,DRIVE_SOLO_TO_WORK_MARG,CARPOOL_TO_WORK_EST,CARPOOL_TO_WORK_MARG,PUBTRANS_TO_WORK_EST,PUBTRANS_TO_WORK_MARG,state,county,county_subdivision,MUNI_NAME,COUNTY_NAME_x,PENN_DOT_COUNTY_NUM,PENN_DOT_COUNTY_NAME,PENN_DOT_MUNI_ID,PENN_DOT_MUNI_NAME,COUNTY_NAME_y,textbox5,MUNICIPALITY_NAME,CENSUS_POPULATION,LAND_AREA,FEDERAL_EIN_CODE,HOME_RULE_CODE,HOME_RULE_NAME,HOME_RULE_YEAR,GOVERNMENTAL_FORM,EMPLOYEES_FULL_TIME,EMPLOYEES_PART_TIME,INCORPORATION_YEAR
0,"Abbottstown borough, Adams County",857,7,11,6,8,355,60,43,22,0,11,42,1,116,ABBOTTSTOWN BOROUGH,ADAMS COUNTY,1,ADAMS COUNTY,"1,401.000000",ABBOTTSTOWN BOROUGH,ADAMS COUNTY,"10,033.000000",ABBOTTSTOWN BOROUGH,"1,022.000000",0.600000,"236,050,408.000000",NaN,NaN,NaN,NaN,1.000000,10.000000,"1,835.000000"
1,"Arendtsville borough, Adams County",720,0,11,4,9,301,92,41,33,0,11,42,1,2928,ARENDTSVILLE BOROUGH,ADAMS COUNTY,1,ADAMS COUNTY,"1,402.000000",ARENDTSVILLE BOROUGH,ADAMS COUNTY,"10,063.000000",ARENDTSVILLE BOROUGH,867.000000,0.800000,"231,603,459.000000",NaN,NaN,NaN,NaN,3.000000,2.000000,"1,896.000000"
2,"Bendersville borough, Adams County",672,0,11,4,5,305,97,6,9,0,11,42,1,5536,BENDERSVILLE BOROUGH,ADAMS COUNTY,1,ADAMS COUNTY,"1,403.000000",BENDERSVILLE BOROUGH,ADAMS COUNTY,"10,093.000000",BENDERSVILLE BOROUGH,736.000000,0.500000,"237,447,802.000000",NaN,NaN,NaN,NaN,0.000000,19.000000,"1,866.000000"


In [17]:
#reorganize columns 
MUNI_DF.rename(columns={'COUNTY_NAME_x':'COUNTY_NAME'}, inplace=True)
#MUNI_DF.drop('COUNTY_NAME_y', inplace=True)


MUNI_DF = MUNI_DF[['NAME','PENN_DOT_MUNI_ID','state',
                     'county','county_subdivision',
                     'POPULATION','LAND_AREA',
                     'BIKE_TO_WORK_EST','BIKE_TO_WORK_MARG',
                     'WALK_TO_WORK_EST','WALK_TO_WORK_MARG',
                     'DRIVE_SOLO_TO_WORK_EST','DRIVE_SOLO_TO_WORK_MARG',
                     'CARPOOL_TO_WORK_EST','CARPOOL_TO_WORK_MARG',
                     'PUBTRANS_TO_WORK_EST','PUBTRANS_TO_WORK_MARG',
                     'MUNI_NAME','COUNTY_NAME','PENN_DOT_COUNTY_NUM',
                     'PENN_DOT_COUNTY_NAME',
                     'PENN_DOT_MUNI_NAME','MUNICIPALITY_NAME',
                     'LAND_AREA','FEDERAL_EIN_CODE','HOME_RULE_CODE',
                     'HOME_RULE_NAME','HOME_RULE_YEAR','GOVERNMENTAL_FORM',
                     'EMPLOYEES_FULL_TIME','EMPLOYEES_PART_TIME',
                     'INCORPORATION_YEAR']]
MUNI_DF.head()

,NAME,PENN_DOT_MUNI_ID,state,county,county_subdivision,POPULATION,LAND_AREA,BIKE_TO_WORK_EST,BIKE_TO_WORK_MARG,WALK_TO_WORK_EST,WALK_TO_WORK_MARG,DRIVE_SOLO_TO_WORK_EST,DRIVE_SOLO_TO_WORK_MARG,CARPOOL_TO_WORK_EST,CARPOOL_TO_WORK_MARG,PUBTRANS_TO_WORK_EST,PUBTRANS_TO_WORK_MARG,MUNI_NAME,COUNTY_NAME,PENN_DOT_COUNTY_NUM,PENN_DOT_COUNTY_NAME,PENN_DOT_MUNI_NAME,MUNICIPALITY_NAME,LAND_AREA,FEDERAL_EIN_CODE,HOME_RULE_CODE,HOME_RULE_NAME,HOME_RULE_YEAR,GOVERNMENTAL_FORM,EMPLOYEES_FULL_TIME,EMPLOYEES_PART_TIME,INCORPORATION_YEAR
0,"Abbottstown borough, Adams County","1,401.000000",42,1,116,857,0.600000,7,11,6,8,355,60,43,22,0,11,ABBOTTSTOWN BOROUGH,ADAMS COUNTY,1,ADAMS COUNTY,ABBOTTSTOWN BOROUGH,ABBOTTSTOWN BOROUGH,0.600000,"236,050,408.000000",NaN,NaN,NaN,NaN,1.000000,10.000000,"1,835.000000"
1,"Arendtsville borough, Adams County","1,402.000000",42,1,2928,720,0.800000,0,11,4,9,301,92,41,33,0,11,ARENDTSVILLE BOROUGH,ADAMS COUNTY,1,ADAMS COUNTY,ARENDTSVILLE BOROUGH,ARENDTSVILLE BOROUGH,0.800000,"231,603,459.000000",NaN,NaN,NaN,NaN,3.000000,2.000000,"1,896.000000"
2,"Bendersville borough, Adams County","1,403.000000",42,1,5536,672,0.500000,0,11,4,5,305,97,6,9,0,11,BENDERSVILLE BOROUGH,ADAMS COUNTY,1,ADAMS COUNTY,BENDERSVILLE BOROUGH,BENDERSVILLE BOROUGH,0.500000,"237,447,802.000000",NaN,NaN,NaN,NaN,0.000000,19.000000,"1,866.000000"
3,"Berwick township, Adams County","1,201.000000",42,1,5880,2495,7.700000,0,11,5,6,961,146,129,64,0,11,BERWICK TOWNSHIP,ADAMS COUNTY,1,ADAMS COUNTY,BERWICK TOWNSHIP,BERWICK TOWNSHIP,7.700000,"232,030,284.000000",NaN,NaN,NaN,NaN,4.000000,0.000000,"1,800.000000"
4,"Biglerville borough, Adams County","1,404.000000",42,1,6296,1209,0.700000,0,11,20,24,471,105,86,96,0,11,BIGLERVILLE BOROUGH,ADAMS COUNTY,1,ADAMS COUNTY,BIGLERVILLE BOROUGH,BIGLERVILLE BOROUGH,0.700000,"232,083,930.000000",NaN,NaN,NaN,NaN,5.000000,16.000000,NaN


In [18]:
MUNI_DF.sort_values(by = 'BIKE_TO_WORK_EST', ascending=False, inplace=True)
MUNI_DF=MUNI_DF.dropna(subset=['PENN_DOT_MUNI_ID'])
MUNI_DF['PENN_DOT_MUNI_ID']=MUNI_DF['PENN_DOT_MUNI_ID'].astype(int)
MUNI_DF.head(3)



,NAME,PENN_DOT_MUNI_ID,state,county,county_subdivision,POPULATION,LAND_AREA,BIKE_TO_WORK_EST,BIKE_TO_WORK_MARG,WALK_TO_WORK_EST,WALK_TO_WORK_MARG,DRIVE_SOLO_TO_WORK_EST,DRIVE_SOLO_TO_WORK_MARG,CARPOOL_TO_WORK_EST,CARPOOL_TO_WORK_MARG,PUBTRANS_TO_WORK_EST,PUBTRANS_TO_WORK_MARG,MUNI_NAME,COUNTY_NAME,PENN_DOT_COUNTY_NUM,PENN_DOT_COUNTY_NAME,PENN_DOT_MUNI_NAME,MUNICIPALITY_NAME,LAND_AREA,FEDERAL_EIN_CODE,HOME_RULE_CODE,HOME_RULE_NAME,HOME_RULE_YEAR,GOVERNMENTAL_FORM,EMPLOYEES_FULL_TIME,EMPLOYEES_PART_TIME,INCORPORATION_YEAR
1974,"Philadelphia city, Philadelphia County",67301,42,101,60000,1596865,134.100000,14172,940,54269,2031,343702,5207,55482,3121,152058,4179,PHILADELPHIA CITY,PHILADELPHIA COUNTY,67,PHILADELPHIA COUNTY,PHILADELPHIA CITY,PHILADELPHIA CITY,134.100000,"236,003,047.000000",Home Rule,PHILADELPHIA CITY,"1,952.000000",NaN,"22,325.000000","1,037.000000",NaN
122,"Pittsburgh city, Allegheny County",2301,42,3,61000,303207,55.400000,1920,290,15005,1033,80593,2054,10191,814,23357,1252,PITTSBURGH CITY,ALLEGHENY COUNTY,2,ALLEGHENY COUNTY,PITTSBURGH CITY,PITTSBURGH CITY,55.400000,"256,000,879.000000",Home Rule,CITY OF PITTSBURGH,"1,976.000000",NaN,"3,098.000000",149.000000,"1,816.000000"
683,"State College borough, Centre County",14410,42,27,73808,40352,4.600000,729,297,3963,402,6711,800,484,166,1813,392,STATE COLLEGE BOROUGH,CENTRE COUNTY,14,CENTRE COUNTY,STATE COLLEGE BOROUGH,STATE COLLEGE BOROUGH,4.600000,"246,000,660.000000",Home Rule,BOROUGH OF STATE COLLEGE,"1,976.000000",NaN,166.000000,23.000000,"1,896.000000"


In [19]:
MUNI_DF['PENN_DOT_MUNI_ID'].nunique()

2569

In [20]:
print(len(MUNI_DF))

2569


In [21]:
MUNI_DF["NAME"][MUNI_DF["POPULATION"].isna()==True].unique()

array([], dtype=object)

# Import, Clean and Pivot Penn DoT Crash Data 
data is pivoted and summed to each municipality 

Severe Injuries, Minor Injuries, crashes 


In [22]:
#Import State Crash Data
#import data pdf

# import for the last year to be sampled 
PA_CRASH_DF = pd.read_csv('../../../Data_Work/data_folder/Crash_Data_PA/PA_CRASH_2021/CRASH_2021_Statewide.csv'.format(END_YEAR))

#import the years starting from the second to last to the start year 
for i in range((END_YEAR-1),START_YEAR,-1):
    newtraffic_df = pd.read_csv('../../../Data_Work/data_folder/Crash_Data_PA/PA_CRASH_{}/CRASH_{}_Statewide.csv'.format(i,i))
    PA_CRASH_DF = PA_CRASH_DF.append(newtraffic_df)
    
print("{} of Crashes between {} and {} in Pennsylvania ".format(len(PA_CRASH_DF),START_YEAR,END_YEAR))
PA_CRASH_DF.head(4)


/home/mitch/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (94,96) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/mitch/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (94,96,97,98) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/mitch/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (97,98) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


476220 of Crashes between 2017 and 2021 in Pennsylvania 


,ARRIVAL_TM,AUTOMOBILE_COUNT,BELTED_DEATH_COUNT,BELTED_SUSP_SERIOUS_INJ_COUNT,BICYCLE_COUNT,BICYCLE_DEATH_COUNT,BICYCLE_SUSP_SERIOUS_INJ_COUNT,BUS_COUNT,CHLDPAS_DEATH_COUNT,CHLDPAS_SUSP_SERIOUS_INJ_COUNT,COLLISION_TYPE,COMM_VEH_COUNT,CONS_ZONE_SPD_LIM,COUNTY,CRASH_MONTH,CRASH_YEAR,CRN,DAY_OF_WEEK,DEC_LAT,DEC_LONG,DISPATCH_TM,DISTRICT,DRIVER_COUNT_16YR,DRIVER_COUNT_17YR,DRIVER_COUNT_18YR,DRIVER_COUNT_19YR,DRIVER_COUNT_20YR,DRIVER_COUNT_50_64YR,DRIVER_COUNT_65_74YR,DRIVER_COUNT_75PLUS,EST_HRS_CLOSED,FATAL_COUNT,HEAVY_TRUCK_COUNT,HORSE_BUGGY_COUNT,HOUR_OF_DAY,ILLUMINATION,INJURY_COUNT,INTERSECT_TYPE,LANE_CLOSED,LATITUDE,LN_CLOSE_DIR,LOCATION_TYPE,LONGITUDE,MAX_SEVERITY_LEVEL,MCYCLE_DEATH_COUNT,MCYCLE_SUSP_SERIOUS_INJ_COUNT,MOTORCYCLE_COUNT,MUNICIPALITY,NONMOTR_COUNT,NONMOTR_DEATH_COUNT,NONMOTR_SUSP_SERIOUS_INJ_COUNT,NTFY_HIWY_MAINT,PED_COUNT,PED_DEATH_COUNT,PED_SUSP_SERIOUS_INJ_COUNT,PERSON_COUNT,POLICE_AGCY,POSSIBLE_INJ_COUNT,RDWY_SURF_TYPE_CD,RELATION_TO_ROAD,ROAD_CONDITION,SCH_BUS_IND,SCH_ZONE_IND,SMALL_TRUCK_COUNT,SPEC_JURIS_CD,SUSP_MINOR_INJ_COUNT,SUSP_SERIOUS_INJ_COUNT,SUV_COUNT,TCD_FUNC_CD,TCD_TYPE,TFC_DETOUR_IND,TIME_OF_DAY,TOT_INJ_COUNT,TOTAL_UNITS,UNB_DEATH_COUNT,UNB_SUSP_SERIOUS_INJ_COUNT,UNBELTED_OCC_COUNT,UNK_INJ_DEG_COUNT,UNK_INJ_PER_COUNT,URBAN_RURAL,VAN_COUNT,VEHICLE_COUNT,WORK_ZONE_IND,WORK_ZONE_LOC,WORK_ZONE_TYPE,WORKERS_PRES,WZ_CLOSE_DETOUR,WZ_FLAGGER,WZ_LAW_OFFCR_IND,WZ_LN_CLOSURE,WZ_MOVING,WZ_OTHER,WZ_SHLDER_MDN,ROADWAY_CLEARED,WEATHER1,WEATHER2,WZ_WORKERS_INJ_KILLED,SECONDARY_CRASH,INTERSECTION_RELATED
0,855.000000,2,0,0,0,0,0,0,0,0,1,0,NaN,9,10,2021,2020085055,4,40.150600,-74.860900,850.000000,6,0,0,0,0,0,0,0,0,NaN,0,0,0.000000,8.000000,1,2,0,2,40 09:02.012,4.000000,0,74 51:39.144,3,0,0,0,9101,0,0,0,N,0,0,0,2,09101,0,NaN,1.000000,9.000000,N,N,0,NaN,2,0,0,3.000000,3,N,849.000000,2,2,0,0,0,0,0,3,0,2,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,015.000000",NaN,4.000000,NaN,N,Y
1,"1,655.000000",1,0,0,0,0,0,0,0,0,1,0,NaN,23,1,2021,2020105662,3,39.917000,-75.265400,"1,652.000000",6,0,0,0,1,0,1,0,0,NaN,0,0,0.000000,16.000000,4,1,0,0,39 55:01.343,NaN,0,75 15:55.443,8,0,0,0,23405,0,0,0,N,0,0,0,2,23405,0,NaN,1.000000,1.000000,N,N,0,NaN,0,0,1,3.000000,2,NaN,"1,652.000000",1,2,0,0,0,1,0,3,0,2,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,N,Y
2,34.000000,0,0,0,0,0,0,0,0,0,7,0,NaN,2,1,2021,2021000002,6,40.452500,-79.844400,30.000000,11,0,0,0,0,0,0,0,0,NaN,0,0,0.000000,0.000000,3,4,0,1,40 27:08.919,1.000000,0,79 50:39.666,4,0,0,0,2113,0,0,0,N,0,0,0,5,02113,1,NaN,5.000000,9.000000,N,N,0,NaN,0,0,1,0.000000,0,N,28.000000,4,1,0,0,2,3,1,3,0,1,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,NaN,3.000000,NaN,N,N
3,111.000000,2,0,0,0,0,0,0,0,0,2,0,NaN,2,1,2021,2021000007,6,40.460200,-79.842600,106.000000,11,0,0,0,0,0,0,0,0,NaN,0,0,0.000000,1.000000,3,0,5,0,40 27:36.835,NaN,0,79 50:33.477,0,0,0,0,2113,0,0,0,N,0,0,0,2,02113,0,NaN,1.000000,1.000000,N,N,0,NaN,0,0,0,3.000000,2,NaN,100.000000,0,2,0,0,0,0,0,3,0,2,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,N,N


In [23]:
PA_CRASH_DF['MUNICIPALITY'][((PA_CRASH_DF['PED_COUNT']>=1)|(PA_CRASH_DF['BICYCLE_COUNT']>=1)) & (PA_CRASH_DF['AUTOMOBILE_COUNT']>0)].nunique()

940

In [24]:
PA_CRASH_DF['MUNICIPALITY'][((PA_CRASH_DF['PED_COUNT']>=1)|(PA_CRASH_DF['BICYCLE_COUNT']>=1)) & (PA_CRASH_DF['AUTOMOBILE_COUNT']==0)].nunique()

1035

In [25]:
PA_CRASH_DF['MUNICIPALITY'][(PA_CRASH_DF['BICYCLE_COUNT']>=1) & (PA_CRASH_DF['AUTOMOBILE_COUNT']>0)].nunique()

448

In [26]:
PA_CRASH_DF['MUNICIPALITY'][(PA_CRASH_DF['PED_COUNT']>=1) & (PA_CRASH_DF['AUTOMOBILE_COUNT']==0)].nunique()

906

In [27]:
PA_CRASH_DF['MUNICIPALITY'][(PA_CRASH_DF['BICYCLE_COUNT']>=1) & (PA_CRASH_DF['AUTOMOBILE_COUNT']==0)].nunique()

496

In [28]:
# select entries where automobiles crashed with cyclists or pedestrians  after 2012
car_bike_and_pedestrian_df=PA_CRASH_DF[((PA_CRASH_DF['PED_COUNT']>=1)|
                                   (PA_CRASH_DF['BICYCLE_COUNT']>=1))&
                                  (PA_CRASH_DF['AUTOMOBILE_COUNT']>0)]

# Sum the crashes and pivot to municipal ID number and create a new DF for merging with Municipal data 
Muni_car_bike_ped_crash_df = pd.pivot_table(car_bike_and_pedestrian_df, values=('BICYCLE_COUNT',
                                                                               'BICYCLE_DEATH_COUNT',
                                                                               'BICYCLE_SUSP_SERIOUS_INJ_COUNT',
                                                                               'AUTOMOBILE_COUNT',
                                                                               'PED_COUNT',
                                                                               'PED_DEATH_COUNT', 
                                                                               'PED_SUSP_SERIOUS_INJ_COUNT'), index='MUNICIPALITY', aggfunc='sum')
                                                                               #'MOTORCYCLE_COUNT',
                                                                               #'SMALL_TRUCK_COUNT',
                                                                               #'SUV_COUNT',
                                                                               #'VAN_COUNT',
                                                                               #'VEHICLE_COUNT'
                                                                           
Muni_car_bike_ped_crash_df.reset_index(inplace=True)

# rename the columns to include BY_AUTO to differentiate between 
Muni_car_bike_ped_crash_df=Muni_car_bike_ped_crash_df.rename(columns={'BICYCLE_COUNT':'BICYCLE_BY_AUTO_COUNT',
                                                                      'BICYCLE_DEATH_COUNT':'BICYCLE_DEATH_BY_AUTO_COUNT',
                                                                      'BICYCLE_SUSP_SERIOUS_INJ_COUNT':'BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT',
                                                                      'PED_COUNT':'PED_BY_AUTO_COUNT',
                                                                      'PED_DEATH_COUNT':'PED_DEATH_BY_AUTO_COUNT', 
                                                                      'PED_SUSP_SERIOUS_INJ_COUNT':'PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT'})

print(len(Muni_car_bike_ped_crash_df))
Muni_car_bike_ped_crash_df.head(3)


940


,MUNICIPALITY,AUTOMOBILE_COUNT,BICYCLE_BY_AUTO_COUNT,BICYCLE_DEATH_BY_AUTO_COUNT,BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT,PED_BY_AUTO_COUNT,PED_DEATH_BY_AUTO_COUNT,PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT
0,1201,1,0,0,0,1,1,0
1,1203,1,0,0,0,1,1,0
2,1204,2,0,0,0,1,0,0


In [29]:
#Select Crashes that did'nt involve Automobiles crashing with cyclists and pedestrians
solo_bike_ped_crash_df=PA_CRASH_DF[((PA_CRASH_DF['PED_COUNT']>=1)|
                                   (PA_CRASH_DF['BICYCLE_COUNT']>=1))&
                                  (PA_CRASH_DF['AUTOMOBILE_COUNT']==0)]

# Sum the crashes and pivot to municipal ID number and create a new DF for merging with Municipal data 
Muni_solo_bike_ped_crash_df = pd.pivot_table(solo_bike_ped_crash_df, values=('BICYCLE_COUNT',
           'BICYCLE_DEATH_COUNT',
           'BICYCLE_SUSP_SERIOUS_INJ_COUNT',
           'PED_COUNT',
           'PED_DEATH_COUNT', 
           'PED_SUSP_SERIOUS_INJ_COUNT'), index='MUNICIPALITY', aggfunc='sum')
Muni_solo_bike_ped_crash_df.reset_index(inplace=True)

# rename the columns to include BY_AUTO to differentiate between 
Muni_solo_bike_ped_crash_df=Muni_solo_bike_ped_crash_df.rename(columns={'BICYCLE_COUNT':'BICYCLE_SOLO_COUNT',
                                                                       'BICYCLE_DEATH_COUNT':'BICYCLE_DEATH_SOLO_COUNT',
                                                                       'BICYCLE_SUSP_SERIOUS_INJ_COUNT':'BICYCLE_SUSP_SERIOUS_INJ_SOLO_COUNT',
                                                                       'PED_COUNT':'PED_SOLO_COUNT',
                                                                       'PED_DEATH_COUNT':'PED_DEATH_SOLO_COUNT', 
                                                                       'PED_SUSP_SERIOUS_INJ_COUNT':'PED_SUSP_SERIOUS_INJ_SOLO_COUNT'})
print(len(Muni_solo_bike_ped_crash_df))
Muni_solo_bike_ped_crash_df.head(3)

1035


,MUNICIPALITY,BICYCLE_SOLO_COUNT,BICYCLE_DEATH_SOLO_COUNT,BICYCLE_SUSP_SERIOUS_INJ_SOLO_COUNT,PED_SOLO_COUNT,PED_DEATH_SOLO_COUNT,PED_SUSP_SERIOUS_INJ_SOLO_COUNT
0,1201,0,0,0,1,0,0
1,1203,1,0,0,1,0,0
2,1207,1,0,1,0,0,0


In [30]:
del(CRASH_DF)


NameError: name 'CRASH_DF' is not defined

# Merge the crash Data and the Municipal Data 

In [31]:
#del(CRASH_DF)

#merge pivoted crash data frames into one 
CRASH_DF=Muni_car_bike_ped_crash_df.merge(
                  Muni_solo_bike_ped_crash_df,
                  how="outer",
                  left_on='MUNICIPALITY',
                  right_on='MUNICIPALITY')

#delete pivoted data frames to free up memory
del(Muni_solo_bike_ped_crash_df,Muni_car_bike_ped_crash_df)

#print the length of the crash data 
print(len(CRASH_DF))
CRASH_DF.head(3)


1309


,MUNICIPALITY,AUTOMOBILE_COUNT,BICYCLE_BY_AUTO_COUNT,BICYCLE_DEATH_BY_AUTO_COUNT,BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT,PED_BY_AUTO_COUNT,PED_DEATH_BY_AUTO_COUNT,PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT,BICYCLE_SOLO_COUNT,BICYCLE_DEATH_SOLO_COUNT,BICYCLE_SUSP_SERIOUS_INJ_SOLO_COUNT,PED_SOLO_COUNT,PED_DEATH_SOLO_COUNT,PED_SUSP_SERIOUS_INJ_SOLO_COUNT
0,1201,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
1,1203,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
2,1204,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
MUNI_DF.columns.tolist()

['NAME',
 'PENN_DOT_MUNI_ID',
 'state',
 'county',
 'county_subdivision',
 'POPULATION',
 'LAND_AREA',
 'BIKE_TO_WORK_EST',
 'BIKE_TO_WORK_MARG',
 'WALK_TO_WORK_EST',
 'WALK_TO_WORK_MARG',
 'DRIVE_SOLO_TO_WORK_EST',
 'DRIVE_SOLO_TO_WORK_MARG',
 'CARPOOL_TO_WORK_EST',
 'CARPOOL_TO_WORK_MARG',
 'PUBTRANS_TO_WORK_EST',
 'PUBTRANS_TO_WORK_MARG',
 'MUNI_NAME',
 'COUNTY_NAME',
 'PENN_DOT_COUNTY_NUM',
 'PENN_DOT_COUNTY_NAME',
 'PENN_DOT_MUNI_NAME',
 'MUNICIPALITY_NAME',
 'LAND_AREA',
 'FEDERAL_EIN_CODE',
 'HOME_RULE_CODE',
 'HOME_RULE_NAME',
 'HOME_RULE_YEAR',
 'GOVERNMENTAL_FORM',
 'EMPLOYEES_FULL_TIME',
 'EMPLOYEES_PART_TIME',
 'INCORPORATION_YEAR']

In [33]:
print(len(MUNI_DF))

2569


In [34]:
CRASH_DF['MUNICIPALITY'].nunique()

1309

In [35]:
print(len(CRASH_DF))

1309


In [36]:
# merge Muicipal data and crash data

MUNI_CRASH_DATA= pd.merge(MUNI_DF,CRASH_DF, how = "outer", left_on= "PENN_DOT_MUNI_ID",right_on="MUNICIPALITY")

print(len(MUNI_DF))

print(len(CRASH_DF))

print(len(MUNI_CRASH_DATA))
MUNI_CRASH_DATA.head(3)


2569
1309
2569


,NAME,PENN_DOT_MUNI_ID,state,county,county_subdivision,POPULATION,LAND_AREA,BIKE_TO_WORK_EST,BIKE_TO_WORK_MARG,WALK_TO_WORK_EST,WALK_TO_WORK_MARG,DRIVE_SOLO_TO_WORK_EST,DRIVE_SOLO_TO_WORK_MARG,CARPOOL_TO_WORK_EST,CARPOOL_TO_WORK_MARG,PUBTRANS_TO_WORK_EST,PUBTRANS_TO_WORK_MARG,MUNI_NAME,COUNTY_NAME,PENN_DOT_COUNTY_NUM,PENN_DOT_COUNTY_NAME,PENN_DOT_MUNI_NAME,MUNICIPALITY_NAME,LAND_AREA,FEDERAL_EIN_CODE,HOME_RULE_CODE,HOME_RULE_NAME,HOME_RULE_YEAR,GOVERNMENTAL_FORM,EMPLOYEES_FULL_TIME,EMPLOYEES_PART_TIME,INCORPORATION_YEAR,MUNICIPALITY,AUTOMOBILE_COUNT,BICYCLE_BY_AUTO_COUNT,BICYCLE_DEATH_BY_AUTO_COUNT,BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT,PED_BY_AUTO_COUNT,PED_DEATH_BY_AUTO_COUNT,PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT,BICYCLE_SOLO_COUNT,BICYCLE_DEATH_SOLO_COUNT,BICYCLE_SUSP_SERIOUS_INJ_SOLO_COUNT,PED_SOLO_COUNT,PED_DEATH_SOLO_COUNT,PED_SUSP_SERIOUS_INJ_SOLO_COUNT
0,"Philadelphia city, Philadelphia County",67301,42,101,60000,1596865,134.100000,14172,940,54269,2031,343702,5207,55482,3121,152058,4179,PHILADELPHIA CITY,PHILADELPHIA COUNTY,67,PHILADELPHIA COUNTY,PHILADELPHIA CITY,PHILADELPHIA CITY,134.100000,"236,003,047.000000",Home Rule,PHILADELPHIA CITY,"1,952.000000",NaN,"22,325.000000","1,037.000000",NaN,"67,301.000000","3,202.000000",582.000000,8.000000,31.000000,"2,608.000000",82.000000,214.000000,505.000000,10.000000,24.000000,"2,655.000000",83.000000,227.000000
1,"Pittsburgh city, Allegheny County",2301,42,3,61000,303207,55.400000,1920,290,15005,1033,80593,2054,10191,814,23357,1252,PITTSBURGH CITY,ALLEGHENY COUNTY,2,ALLEGHENY COUNTY,PITTSBURGH CITY,PITTSBURGH CITY,55.400000,"256,000,879.000000",Home Rule,CITY OF PITTSBURGH,"1,976.000000",NaN,"3,098.000000",149.000000,"1,816.000000","2,301.000000",616.000000,83.000000,1.000000,4.000000,539.000000,6.000000,75.000000,54.000000,1.000000,8.000000,332.000000,16.000000,43.000000
2,"State College borough, Centre County",14410,42,27,73808,40352,4.600000,729,297,3963,402,6711,800,484,166,1813,392,STATE COLLEGE BOROUGH,CENTRE COUNTY,14,CENTRE COUNTY,STATE COLLEGE BOROUGH,STATE COLLEGE BOROUGH,4.600000,"246,000,660.000000",Home Rule,BOROUGH OF STATE COLLEGE,"1,976.000000",NaN,166.000000,23.000000,"1,896.000000","14,410.000000",61.000000,14.000000,0.000000,2.000000,47.000000,0.000000,3.000000,12.000000,0.000000,3.000000,42.000000,0.000000,3.000000


In [37]:
# merge Muicipal 

In [40]:
len(MUNI_CRASH_DATA)

2569

# Save the Data Frame as a CSV file 

In [38]:
# save the Dataframe to a file 

#create the file path and label it with the start and end years
filepath = Path('/home/mitch/Documents/Climate Infographics/{}_TO_{}_MUNI_CRASH_DATA.csv'.format(START_YEAR,END_YEAR))

filepath.parent.mkdir(parents=True, exist_ok=True)  

save_file=1

#save the Data frame as a CSV
if save_file==1:
    MUNI_CRASH_DATA.to_csv(filepath)  
    
else:
    None

#show the dataframe
MUNI_CRASH_DATA.head(30)

,NAME,PENN_DOT_MUNI_ID,state,county,county_subdivision,POPULATION,LAND_AREA,BIKE_TO_WORK_EST,BIKE_TO_WORK_MARG,WALK_TO_WORK_EST,WALK_TO_WORK_MARG,DRIVE_SOLO_TO_WORK_EST,DRIVE_SOLO_TO_WORK_MARG,CARPOOL_TO_WORK_EST,CARPOOL_TO_WORK_MARG,PUBTRANS_TO_WORK_EST,PUBTRANS_TO_WORK_MARG,MUNI_NAME,COUNTY_NAME,PENN_DOT_COUNTY_NUM,PENN_DOT_COUNTY_NAME,PENN_DOT_MUNI_NAME,MUNICIPALITY_NAME,LAND_AREA,FEDERAL_EIN_CODE,HOME_RULE_CODE,HOME_RULE_NAME,HOME_RULE_YEAR,GOVERNMENTAL_FORM,EMPLOYEES_FULL_TIME,EMPLOYEES_PART_TIME,INCORPORATION_YEAR,MUNICIPALITY,AUTOMOBILE_COUNT,BICYCLE_BY_AUTO_COUNT,BICYCLE_DEATH_BY_AUTO_COUNT,BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT,PED_BY_AUTO_COUNT,PED_DEATH_BY_AUTO_COUNT,PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT,BICYCLE_SOLO_COUNT,BICYCLE_DEATH_SOLO_COUNT,BICYCLE_SUSP_SERIOUS_INJ_SOLO_COUNT,PED_SOLO_COUNT,PED_DEATH_SOLO_COUNT,PED_SUSP_SERIOUS_INJ_SOLO_COUNT
0,"Philadelphia city, Philadelphia County",67301,42,101,60000,1596865,134.100000,14172,940,54269,2031,343702,5207,55482,3121,152058,4179,PHILADELPHIA CITY,PHILADELPHIA COUNTY,67,PHILADELPHIA COUNTY,PHILADELPHIA CITY,PHILADELPHIA CITY,134.100000,"236,003,047.000000",Home Rule,PHILADELPHIA CITY,"1,952.000000",NaN,"22,325.000000","1,037.000000",NaN,"67,301.000000","3,202.000000",582.000000,8.000000,31.000000,"2,608.000000",82.000000,214.000000,505.000000,10.000000,24.000000,"2,655.000000",83.000000,227.000000
1,"Pittsburgh city, Allegheny County",2301,42,3,61000,303207,55.400000,1920,290,15005,1033,80593,2054,10191,814,23357,1252,PITTSBURGH CITY,ALLEGHENY COUNTY,2,ALLEGHENY COUNTY,PITTSBURGH CITY,PITTSBURGH CITY,55.400000,"256,000,879.000000",Home Rule,CITY OF PITTSBURGH,"1,976.000000",NaN,"3,098.000000",149.000000,"1,816.000000","2,301.000000",616.000000,83.000000,1.000000,4.000000,539.000000,6.000000,75.000000,54.000000,1.000000,8.000000,332.000000,16.000000,43.000000
2,"State College borough, Centre County",14410,42,27,73808,40352,4.600000,729,297,3963,402,6711,800,484,166,1813,392,STATE COLLEGE BOROUGH,CENTRE COUNTY,14,CENTRE COUNTY,STATE COLLEGE BOROUGH,STATE COLLEGE BOROUGH,4.600000,"246,000,660.000000",Home Rule,BOROUGH OF STATE COLLEGE,"1,976.000000",NaN,166.000000,23.000000,"1,896.000000","14,410.000000",61.000000,14.000000,0.000000,2.000000,47.000000,0.000000,3.000000,12.000000,0.000000,3.000000,42.000000,0.000000,3.000000
3,"Erie city, Erie County",25302,42,49,24000,95536,19.100000,329,191,2003,392,28933,1152,5476,668,1482,312,ERIE CITY,ERIE COUNTY,25,ERIE COUNTY,ERIE CITY,ERIE CITY,19.100000,"256,000,857.000000",Opional 3rd Class City Charter,CITY OF ERIE,"1,962.000000",Mayor/Council,659.000000,78.000000,"1,851.000000","25,302.000000",146.000000,46.000000,1.000000,5.000000,100.000000,3.000000,10.000000,52.000000,0.000000,1.000000,94.000000,5.000000,12.000000
4,"Ferguson township, Centre County",14206,42,27,25624,19236,47.700000,259,116,335,142,6363,680,1169,336,591,211,FERGUSON TOWNSHIP,CENTRE COUNTY,14,CENTRE COUNTY,FERGUSON TOWNSHIP,FERGUSON TOWNSHIP,47.700000,"251,197,270.000000",Home Rule,TOWNSHIP OF FERGUSON,"1,976.000000",NaN,65.000000,9.000000,"1,801.000000","14,206.000000",11.000000,5.000000,0.000000,2.000000,4.000000,0.000000,0.000000,3.000000,1.000000,0.000000,3.000000,0.000000,1.000000
5,"Carlisle borough, Cumberland County",21402,42,41,11272,19869,5.500000,235,104,1052,222,6598,481,547,183,42,37,CARLISLE BOROUGH,CUMBERLAND COUNTY,21,CUMBERLAND COUNTY,CARLISLE BOROUGH,CARLISLE BOROUGH,5.500000,"236,002,842.000000",NaN,NaN,NaN,NaN,109.000000,19.000000,"1,782.000000","21,402.000000",42.000000,18.000000,0.000000,4.000000,24.000000,0.000000,3.000000,12.000000,0.000000,2.000000,25.000000,1.000000,3.000000
6,"Ephrata township, Lancaster County",36218,42,71,23840,10375,16.200000,225,148,192,152,3579,400,932,710,1,2,EPHRATA TOWNSHIP,LANCASTER COUNTY,36,LANCASTER COUNTY,EPHRATA TOWNSHIP,EPHRATA TOWNSHIP,16.200000,"236,000,326.000000",NaN,NaN,NaN,NaN,6.000000,26.000000,"1,838.000000","36,218.000000",3.000000,0.000000,0.000000,0.000000,3.000000,0.000000,1.000000,3.

In [40]:
MUNI_CRASH_DATA.head()

,NAME,PENN_DOT_MUNI_ID,state,county,county_subdivision,POPULATION,LAND_AREA,BIKE_TO_WORK_EST,BIKE_TO_WORK_MARG,WALK_TO_WORK_EST,WALK_TO_WORK_MARG,DRIVE_SOLO_TO_WORK_EST,DRIVE_SOLO_TO_WORK_MARG,CARPOOL_TO_WORK_EST,CARPOOL_TO_WORK_MARG,PUBTRANS_TO_WORK_EST,PUBTRANS_TO_WORK_MARG,MUNI_NAME,COUNTY_NAME,PENN_DOT_COUNTY_NUM,PENN_DOT_COUNTY_NAME,PENN_DOT_MUNI_NAME,MUNICIPALITY_NAME,LAND_AREA,FEDERAL_EIN_CODE,HOME_RULE_CODE,HOME_RULE_NAME,HOME_RULE_YEAR,GOVERNMENTAL_FORM,EMPLOYEES_FULL_TIME,EMPLOYEES_PART_TIME,INCORPORATION_YEAR,MUNICIPALITY,AUTOMOBILE_COUNT,BICYCLE_BY_AUTO_COUNT,BICYCLE_DEATH_BY_AUTO_COUNT,BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT,PED_BY_AUTO_COUNT,PED_DEATH_BY_AUTO_COUNT,PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT,BICYCLE_SOLO_COUNT,BICYCLE_DEATH_SOLO_COUNT,BICYCLE_SUSP_SERIOUS_INJ_SOLO_COUNT,PED_SOLO_COUNT,PED_DEATH_SOLO_COUNT,PED_SUSP_SERIOUS_INJ_SOLO_COUNT
0,"Philadelphia city, Philadelphia County","67,301.000000",42.000000,101.000000,"60,000.000000","1,596,865.000000",134.100000,"14,172.000000",940.000000,"54,269.000000","2,031.000000","343,702.000000","5,207.000000","55,482.000000","3,121.000000","152,058.000000","4,179.000000",PHILADELPHIA CITY,PHILADELPHIA COUNTY,67.000000,PHILADELPHIA COUNTY,PHILADELPHIA CITY,PHILADELPHIA CITY,134.100000,"236,003,047.000000",Home Rule,PHILADELPHIA CITY,"1,952.000000",NaN,"22,325.000000","1,037.000000",NaN,"67,301.000000","9,590.000000","2,091.000000",13.000000,67.000000,"7,563.000000",180.000000,427.000000,"1,551.000000",20.000000,57.000000,"6,279.000000",167.000000,390.000000
1,"Pittsburgh city, Allegheny County","2,301.000000",42.000000,3.000000,"61,000.000000","303,207.000000",55.400000,"1,920.000000",290.000000,"15,005.000000","1,033.000000","80,593.000000","2,054.000000","10,191.000000",814.000000,"23,357.000000","1,252.000000",PITTSBURGH CITY,ALLEGHENY COUNTY,2.000000,ALLEGHENY COUNTY,PITTSBURGH CITY,PITTSBURGH CITY,55.400000,"256,000,879.000000",Home Rule,CITY OF PITTSBURGH,"1,976.000000",NaN,"3,098.000000",149.000000,"1,816.000000","2,301.000000","1,588.000000",266.000000,3.000000,5.000000,"1,325.000000",14.000000,144.000000,172.000000,3.000000,13.000000,855.000000,33.000000,76.000000
2,"State College borough, Centre County","14,410.000000",42.000000,27.000000,"73,808.000000","40,352.000000",4.600000,729.000000,297.000000,"3,963.000000",402.000000,"6,711.000000",800.000000,484.000000,166.000000,"1,813.000000",392.000000,STATE COLLEGE BOROUGH,CENTRE COUNTY,14.000000,CENTRE COUNTY,STATE COLLEGE BOROUGH,STATE COLLEGE BOROUGH,4.600000,"246,000,660.000000",Home Rule,BOROUGH OF STATE COLLEGE,"1,976.000000",NaN,166.000000,23.000000,"1,896.000000","14,410.000000",172.000000,50.000000,1.000000,3.000000,127.000000,2.000000,7.000000,31.000000,0.000000,5.000000,94.000000,1.000000,8.000000
3,"Erie city, Erie County","25,302.000000",42.000000,49.000000,"24,000.000000","95,536.000000",19.100000,329.000000,191.000000,"2,003.000000",392.000000,"28,933.000000","1,152.000000","5,476.000000",668.000000,"1,482.000000",312.000000,ERIE CITY,ERIE COUNTY,25.000000,ERIE COUNTY,ERIE CITY,ERIE CITY,19.100000,"256,000,857.000000",Opional 3rd Class City Charter,CITY OF ERIE,"1,962.000000",Mayor/Council,659.000000,78.000000,"1,851.000000","25,302.000000",369.000000,113.000000,1.000000,8.000000,258.000000,8.000000,27.000000,117.000000,1.000000,4.000000,241.000000,7.000000,29.000000
4,"Ferguson township, Centre County","14,206.000000",42.000000,27.000000,"25,624.000000","19,236.000000",47.700000,259.000000,116.000000,335.000000,142.000000,"6,363.000000",680.000000,"1,169.000000",336.000000,591.000000,211.000000,FERGUSON TOWNSHIP,CENTRE COUNTY,14.000000,CENTRE COUNTY,FERGUSON TOWNSHIP,FERGUSON TOWNSHIP,47.700000,"251,197,270.000000",Home Rule,TOWNSHIP OF FERGUSON,"1,976.000000",NaN,65.000000,9.000000,"1,801.000000","14,206.000000",22.000000,9.000000,0.000000,2.000000,10.000000,0.000000,0.000000,8.000000,1.000000,1.000000,8.000000,0.000000,3.000000
